In [2]:
%cd /content/drive/MyDrive/project FL

/content/drive/MyDrive/project FL


In [1]:
import os
import pandas as pd
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

# Load the pre-trained CLIP model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Path to the CSV file
csv_path = "/content/drive/MyDrive/project FL/input_comic.xlsx"

# Load the CSV file into a DataFrame
df = pd.read_excel(csv_path)

# Function to calculate CLIP-I and CLIP-T scores between text and image
def calculate_clip_scores(prompt, image_path):
    # Load image using PIL
    image = Image.open(image_path)

    # Prepare inputs
    inputs = processor(text=[prompt], images=image, return_tensors="pt", padding=True)

    # Forward pass through the model
    outputs = model(**inputs)

    # Extract image and text embeddings
    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds

    # Normalize embeddings
    image_embeds = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
    text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)

    # Compute cosine similarity as the CLIP score

    clip_t_score = (text_embeds @ image_embeds.T).squeeze()

    return  clip_t_score.item()

# Iterate through each row in the DataFrame and calculate CLIP-I and CLIP-T scores
for idx, row in df.iterrows():
    for i in range(1, 31):
        prompt = row.get(f'prompt_{i}')
        image_path = row.get(f'p_file_path_{i}')
        if pd.notna(prompt) and pd.notna(image_path) and os.path.exists(image_path):
            try:
                clip_i_score, clip_t_score = calculate_clip_scores(prompt, image_path)

                df.at[idx, f'score_t_{i}'] = clip_t_score
            except Exception as e:
                print(f"Error processing row {idx}, panel {i}: {e}")

# Save the updated DataFrame back to CSV
df.to_excel("CLIP_T_scores.xlsx", index=False)

print("CLIP-I and CLIP-T scores calculated and saved to CLIP_T_scores.xlsx")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIP-I and CLIP-T scores calculated and saved to CLIP_T_scores.xlsx


In [3]:
import os
import dlib
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine
import cv2

# Load Dlib's face detector and the face recognition model
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
face_rec_model = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
similarity_score = ""

# Function to get face embeddings from an image
def get_face_embedding(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise Exception(f"Image not found or unable to read: {image_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB as Dlib expects RGB images
    faces = face_detector(img, 1)  # Adding upsample_num_times to 1 to improve face detection
    if len(faces) == 0:
        return None  # Return None if no face is detected
    shape = shape_predictor(img, faces[0])
    embedding = face_rec_model.compute_face_descriptor(img, shape)
    return np.array(embedding)

# Function to check cosine similarity between input image and panel images
def check_face_similarities(input_image_path, panel_images):
    input_embedding = get_face_embedding(input_image_path)
    if input_embedding is None:
        print(f"No face detected in input image: {input_image_path}")
        return 0  # Return 0 if no face detected in input image

    max_similarity_score = 0
    for panel_image_path in panel_images:
        try:
            panel_embedding = get_face_embedding(panel_image_path)
            if panel_embedding is None:
                print(f"No face detected in panel image: {panel_image_path}")
                similarity_score = 0  # Return 0 if no face detected in panel image
            else:
                similarity_score = 1 - cosine(input_embedding, panel_embedding)
            print(f"Cosine similarity between {os.path.basename(input_image_path)} and {os.path.basename(panel_image_path)}: {similarity_score}")
            max_similarity_score = max(max_similarity_score, similarity_score)
        except Exception as e:
            print(f"Error processing {panel_image_path}: {e}")
    return max_similarity_score


# Load CSV file to read image paths
csv_file_path = "/content/drive/MyDrive/project FL/input_comic.xlsx"

# Load the CSV file into a DataFrame
df = pd.read_excel(csv_file_path)

for idx, row in df.iterrows():
    final_score =[]
    for i in range(1, 7):
        strip_image_path = row.get(f'strip_path')
        input_image_path = strip_image_path.replace('strip.png', 'input.png')
        panel_image_path = row.get(f'p_file_path_{i}')
        print(input_image_path,panel_image_path)
        similarity_score = check_face_similarities(input_image_path, [panel_image_path])
        final_score.append(similarity_score)
        # cosine_score = check_face_similarities(input_image_path,panel_image_path)


    # Store the list of similarity scores as a string in the DataFrame
    df.at[idx, 'Cosine similarity'] = ', '.join(map(str, final_score))



# # Save the updated DataFrame back to CSV
df.to_excel("Cosine similarity_scores.xlsx", index=False)

print("Cosine similarity calculated and saved to Cosine similarity_scores.xlsx")






outputs/Neha the Kite Maker/input.png outputs/Neha the Kite Maker/panel-0.png
No face detected in input image: outputs/Neha the Kite Maker/input.png
outputs/Neha the Kite Maker/input.png outputs/Neha the Kite Maker/panel-1.png
No face detected in input image: outputs/Neha the Kite Maker/input.png
outputs/Neha the Kite Maker/input.png outputs/Neha the Kite Maker/panel-2.png
No face detected in input image: outputs/Neha the Kite Maker/input.png
outputs/Neha the Kite Maker/input.png outputs/Neha the Kite Maker/panel-3.png
No face detected in input image: outputs/Neha the Kite Maker/input.png
outputs/Neha the Kite Maker/input.png outputs/Neha the Kite Maker/panel-4.png
No face detected in input image: outputs/Neha the Kite Maker/input.png
outputs/Neha the Kite Maker/input.png outputs/Neha the Kite Maker/panel-5.png
No face detected in input image: outputs/Neha the Kite Maker/input.png
outputs/Arjun the Potter’s Apprentice/input.png outputs/Arjun the Potter’s Apprentice/panel-0.png
Cosine s